### TOPIS 교통량 데이터를 강남에 한해서 엑셀 파일 수정 및 병합하는 코드

강남 섹터만 골라 필터링 하는 코드

In [6]:
import os
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm  # 진행 상태 표시를 위해 사용
from datetime import datetime, timedelta
import pandas as pd

In [20]:
def filter_traffic_data_all_months():
    # 원하는 지점번호 리스트
    required_points = [
        "B-13", "C-13", "C-17", "D-35", "D-38",
        "D-39", "D-40", "D-42", "D-43", "F-07"
    ]
    
    # 1월부터 11월까지 파일 이름, 시트 이름 처리
    # 2021 : 07월 서울시 교통량 조사자료
    for month in range(1, 13):  # 1부터 12까지
        # 파일 이름 설정
        input_file = f"{month:02d}월 서울시 교통량 조사자료.xlsx"  # 예: 01월 서울시 교통량 조사자료(2024).xlsx
        
        # 시트 이름 설정
        sheet_name = f"2021년 {month:02d}월"  # 예: 2024년 01월
        
        try:
            # 엑셀 파일 읽기
            df = pd.read_excel(input_file, sheet_name=sheet_name)
            
            # 지점번호 필터링
            filtered_df = df[df['지점번호'].isin(required_points)]
            
            # 결과를 저장할 파일 이름 설정
            output_file = f"필터링_서울시_교통량_2021_{month:02d}월.xlsx"
            
            # 필터링된 데이터 저장
            filtered_df.to_excel(output_file, index=False)
            
            print(f"'{input_file}'의 필터링 결과를 '{output_file}'로 저장했습니다.")
        
        except Exception as e:
            print(f"파일 '{input_file}' 또는 시트 '{sheet_name}' 처리 중 오류 발생: {e}")

# 실행
filter_traffic_data_all_months()


'03월 서울시 교통량 조사자료.xlsx'의 필터링 결과를 '필터링_서울시_교통량_2021_03월.xlsx'로 저장했습니다.


병합 코드

In [21]:
def merge_filtered_files():
    # 병합할 파일들이 있는 디렉터리 설정 (현재 디렉터리 기준)
    directory = "./"  # 필터링된 파일들이 있는 디렉터리
    
    # 병합 대상 파일 이름 형식
    file_pattern = "필터링_서울시_교통량_2021_*.xlsx"
    
    # 병합할 데이터프레임을 담을 리스트
    merged_data = []
    
    # 디렉터리에서 파일 검색 및 읽기
    for month in range(1, 12):
        file_name = f"필터링_서울시_교통량_2021_{month:02d}월.xlsx"
        file_path = os.path.join(directory, file_name)
        
        try:
            # 파일 읽기
            df = pd.read_excel(file_path)
            
            # 월 정보 추가 (추후 병합 후 구분 가능하도록)
            df["월"] = f"{month:02d}월"
            
            # 리스트에 추가
            merged_data.append(df)
            
            print(f"파일 '{file_name}' 읽기 성공")
        except Exception as e:
            print(f"파일 '{file_name}' 읽기 실패: {e}")
    
    # 리스트에 저장된 모든 데이터프레임 병합
    if merged_data:
        final_df = pd.concat(merged_data, ignore_index=True)
        
        # 병합 결과를 엑셀로 저장
        output_file = "병합된_서울시_교통량_2021.xlsx"
        final_df.to_excel(output_file, index=False)
        
        print(f"병합된 데이터를 '{output_file}'로 저장했습니다.")
    else:
        print("병합할 파일이 없습니다.")

# 실행
merge_filtered_files()


파일 '필터링_서울시_교통량_2021_01월.xlsx' 읽기 성공
파일 '필터링_서울시_교통량_2021_02월.xlsx' 읽기 성공
파일 '필터링_서울시_교통량_2021_03월.xlsx' 읽기 성공
파일 '필터링_서울시_교통량_2021_04월.xlsx' 읽기 성공
파일 '필터링_서울시_교통량_2021_05월.xlsx' 읽기 성공
파일 '필터링_서울시_교통량_2021_06월.xlsx' 읽기 성공
파일 '필터링_서울시_교통량_2021_07월.xlsx' 읽기 성공
파일 '필터링_서울시_교통량_2021_08월.xlsx' 읽기 성공
파일 '필터링_서울시_교통량_2021_09월.xlsx' 읽기 성공
파일 '필터링_서울시_교통량_2021_10월.xlsx' 읽기 성공
파일 '필터링_서울시_교통량_2021_11월.xlsx' 읽기 성공
병합된 데이터를 '병합된_서울시_교통량_2021.xlsx'로 저장했습니다.


### 데이터 살펴보기

In [13]:
df = pd.read_excel('병합된_서울시_교통량_2024.xlsx')
null_counts = df.groupby('지점번호').apply(lambda group: group.isnull().sum())

# 지점번호 별 총 null값 확인
traffic_columns = [f"{hour}시" for hour in range(24)]  # '0시', '1시', ..., '23시'

# 지점번호별 null값 개수 구하기
traffic_null_counts = df.groupby('지점번호')[traffic_columns].apply(lambda group: group.isnull().sum(axis=1))

# 결과 출력
print("지점번호별 시간대 교통량의 null값 개수:")
print(traffic_null_counts)

print("지점번호 별 NULL값 개수:")
print(null_counts)

지점번호별 시간대 교통량의 null값 개수:
지점번호      
B-13  0       24
      1       24
      2       24
      3       24
      4       24
              ..
F-07  6695     0
      6696     0
      6697     0
      6698     0
      6699     0
Length: 6700, dtype: int64
지점번호 별 NULL값 개수:
      일자  요일  지점명  지점번호  방향  구분   0시   1시   2시   3시  ...  15시  16시  17시  18시  \
지점번호                                                 ...                       
B-13   0   0    0     0   0   0  172  172  172  172  ...  168  168  168  168   
C-13   0   0    0     0   0   0   92   93   93   93  ...   86   86   89   88   
C-17   0   0    0     0   0   0    2    2    3    3  ...    8    8    8    5   
D-35   0   0    0     0   0   0    9    9    9    9  ...    6    6    6    5   
D-38   0   0    0     0   0   0   32   32   33   27  ...   12   12   14   16   
D-39   0   0    0     0   0   0   82   82   82   82  ...   87   87   87   87   
D-40   0   0    0     0   0   0  116  116  116  118  ...  124  125  125  124   
D-42   0   0 

In [30]:
# 지점 번호 별 총 null값 개수

# '0시'부터 '23시'까지 열 선택
traffic_columns = [f"{hour}시" for hour in range(24)]  # '0시', '1시', ..., '23시'

# 지점번호별 null값 총합 계산
traffic_null_sums = df.groupby('지점번호')[traffic_columns].apply(lambda group: group.isnull().sum().sum())
traffic_sums = df.groupby('지점번호')[traffic_columns].apply(lambda group: group.notna().sum().sum())

# 결과 출력
print("지점번호별 null값 총합:")
print(traffic_null_sums)

print("지점번호별 유효값 총합:")
print(traffic_sums)


지점번호별 null값 총합:
지점번호
B-13     4068
C-13     2216
C-17      181
D-35      177
D-38      517
D-39     2069
D-40     2924
D-42    16080
D-43       40
F-07       16
dtype: int64
지점번호별 유효값 총합:
지점번호
B-13    12012
C-13    13864
C-17    15899
D-35    15903
D-38    15563
D-39    14011
D-40    13156
D-42        0
D-43    16040
F-07    16064
dtype: int64


### 공휴일 API로 연도, 월별 공휴일 정보 받아오기

In [ ]:
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
params ={'serviceKey' : '5Cv0uRzvotQnWklDz5xY6cia7S1J+luzfZ4YJTQedGe+F5YeLP6bCUtdx/fbZMjBp4DBeU3b0hQBspFf6EIKOA==',
         'solYear' : '2021', 'solMonth' : '09' }

response = requests.get(url, params=params)
print(response.content)


b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><dateKind>01</dateKind><dateName>\xec\xb6\x94\xec\x84\x9d</dateName><isHoliday>Y</isHoliday><locdate>20210920</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>\xec\xb6\x94\xec\x84\x9d</dateName><isHoliday>Y</isHoliday><locdate>20210921</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>\xec\xb6\x94\xec\x84\x9d</dateName><isHoliday>Y</isHoliday><locdate>20210922</locdate><seq>1</seq></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>3</totalCount></body></response>'


In [39]:
# API 설정
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
service_key = '5Cv0uRzvotQnWklDz5xY6cia7S1J+luzfZ4YJTQedGe+F5YeLP6bCUtdx/fbZMjBp4DBeU3b0hQBspFf6EIKOA=='

# 결과를 저장할 리스트
holidays = []

# 2021년 1월부터 2024년 12월까지 반복
for year in range(2021, 2025):
    for month in range(1, 13):
        # 월을 두 자리 문자열로 변환 (예: 1 -> '01')
        sol_month = f'{month:02d}'
        params = {
            'serviceKey': service_key,
            'solYear': str(year),
            'solMonth': sol_month
        }
        
        # API 요청
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            # XML 파싱
            root = ET.fromstring(response.content)
            items = root.find('.//items')  # items 태그 찾기
            
            if items is not None:
                for item in items.findall('item'):
                    is_holiday = item.find('isHoliday').text
                    if is_holiday == 'Y':  # 공휴일인 경우
                        locdate = item.find('locdate').text  # 날짜
                        holidays.append(locdate)
        else:
            print(f"Error: {response.status_code} for {year}-{sol_month}")

# 결과 출력
print("공휴일 목록:")
for holiday in holidays:
    print(holiday)

# 결과 저장 (선택)
with open('holidays_2021_2024.txt', 'w') as file:
    for holiday in holidays:
        file.write(holiday + '\n')


공휴일 목록:
20210101
20210211
20210212
20210213
20210301
20210505
20210519
20210606
20210815
20210816
20210920
20210921
20210922
20211003
20211004
20211009
20211011
20211225
20220101
20220131
20220201
20220202
20220301
20220309
20220505
20220508
20220601
20220606
20220815
20220909
20220910
20220911
20220912
20221003
20221009
20221010
20221225
20230101
20230121
20230122
20230123
20230124
20230301
20230505
20230527
20230529
20230606
20230815
20230928
20230929
20230930
20231002
20231003
20231009
20231225
20240101
20240209
20240210
20240211
20240212
20240301
20240410
20240505
20240506
20240515
20240606
20240815
20240916
20240917
20240918
20241001
20241003
20241009
20241225


In [43]:
# API 설정
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
service_key = '5Cv0uRzvotQnWklDz5xY6cia7S1J+luzfZ4YJTQedGe+F5YeLP6bCUtdx/fbZMjBp4DBeU3b0hQBspFf6EIKOA=='

# 결과를 저장할 리스트
holiday_data = []

# 2021년 1월부터 2024년 12월까지 반복
for year in tqdm(range(2021, 2025), desc="Processing Years"):
    for month in range(1, 13):
        sol_month = f'{month:02d}'  # 월을 두 자리 문자열로 변환
        params = {
            'serviceKey': service_key,
            'solYear': str(year),
            'solMonth': sol_month
        }
        
        # API 요청
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            # XML 파싱
            root = ET.fromstring(response.content)
            items = root.find('.//items')  # items 태그 찾기
            
            if items is not None:
                for item in items.findall('item'):
                    locdate = item.find('locdate').text  # 날짜
                    date_name = item.find('dateName').text  # 휴일 이름
                    date_kind = item.find('dateKind').text  # 날짜 유형
                    is_holiday = item.find('isHoliday').text  # 공휴일 여부
                    seq = item.find('seq').text  # 순번
                    
                    # 결과 저장
                    holiday_data.append({
                        'locdate': locdate,
                        'seq': int(seq),
                        'dateKind': int(date_kind),
                        'isHoliday': is_holiday == 'Y',  # Boolean 변환
                        'dateName': date_name
                    })
        else:
            print(f"Error: {response.status_code} for {year}-{sol_month}")

# DataFrame 생성
df_holidays = pd.DataFrame(holiday_data)

# 컬럼 데이터 타입 변환
df_holidays['locdate'] = pd.to_datetime(df_holidays['locdate'])  # datetime 형식 변환
df_holidays['seq'] = df_holidays['seq'].astype(int)  # Integer 변환
df_holidays['dateKind'] = df_holidays['dateKind'].astype(int)  # Integer 변환
df_holidays['isHoliday'] = df_holidays['isHoliday'].astype(bool)  # Boolean 변환

# Excel 파일로 저장
df_holidays.to_excel('holidays_2021_2024_formatted.xlsx', index=False, engine='openpyxl')
print("Excel 파일 저장 완료: holidays_2021_2024_formatted.xlsx")


Processing Years: 100%|██████████| 4/4 [00:04<00:00,  1.08s/it]

Excel 파일 저장 완료: holidays_2021_2024_formatted.xlsx


### 지하철 역별/시간대별 이용량 조회 API

In [5]:
def collect_subway_data(api_key, start_date, end_date, locations):
    """
    Collect subway usage data for specified stations over a date range
    
    Parameters:
    api_key (str): Seoul Open API key
    start_date (str): Start date in YYYYMM format
    end_date (str): End date in YYYYMM format
    locations (list): List of station names to filter
    
    Returns:
    pandas.DataFrame: Combined data for all months
    """
    base_url = f'http://openapi.seoul.go.kr:8088/{api_key}/json/CardSubwayTime/1/1000/'
    
    # Convert dates to datetime objects for iteration
    start = datetime.strptime(start_date, '%Y%m')
    end = datetime.strptime(end_date, '%Y%m')
    
    all_data = []
    current = start
    
    while current <= end:
        # Format current date for API call
        current_date = current.strftime('%Y%m')
        
        try:
            # Make API request
            url = base_url + current_date + '/'
            response = requests.get(url)
            result = response.json()
            
            # Convert to DataFrame
            monthly_data = pd.DataFrame(result['CardSubwayTime']['row'])
            
            # Filter for specified stations and create a copy
            filtered_df = monthly_data[monthly_data['STTN'].isin(locations)].copy()
            
            # Add month identifier using loc
            filtered_df.loc[:, 'YEAR_MONTH'] = current_date
            
            # Append to list
            all_data.append(filtered_df)
            
            print(f"Successfully collected data for {current_date}")
            
        except Exception as e:
            print(f"Error collecting data for {current_date}: {str(e)}")
        
        # Move to next month
        current = current + timedelta(days=32)
        current = datetime(current.year, current.month, 1)
    
    # Combine all monthly data
    combined_df = pd.concat(all_data, ignore_index=True)
    
    return combined_df

# Your API key and locations
api_key = '4a4e4a6b6672686b36374f65445347'
locations = [
    "압구정", "신사", "논현", "학동", "강남구청", "청담", "봉은사", "삼성중앙", 
    "선정릉", "언주", "신논현", "강남", "역삼", "선릉", "삼성(무역센터)", 
    "학여울", "대청", "일원", "수서", "대치", "도곡", "매봉", "양재(서초구청)", 
    "구룡", "개포동", "대모산입구", "한티", "압구정로데오"
]

# Collect data from January 2021 to December 2024
combined_data = collect_subway_data(api_key, '202101', '202412', locations)

# Save combined data
combined_data.to_excel("2강남지하철이용량_전체.xlsx", index=True)

# Save individual monthly files if needed
# for year_month in combined_data['YEAR_MONTH'].unique():
    # monthly_data = combined_data[combined_data['YEAR_MONTH'] == year_month].copy()
    # monthly_data.to_excel(f"강남지하철이용량_{year_month}.xlsx", index=True)

Successfully collected data for 202101
Successfully collected data for 202102
Successfully collected data for 202103
Successfully collected data for 202104
Successfully collected data for 202105
Successfully collected data for 202106
Successfully collected data for 202107
Successfully collected data for 202108
Successfully collected data for 202109
Successfully collected data for 202110
Successfully collected data for 202111
Successfully collected data for 202112
Successfully collected data for 202201
Successfully collected data for 202202
Successfully collected data for 202203
Successfully collected data for 202204
Successfully collected data for 202205
Successfully collected data for 202206
Successfully collected data for 202207
Successfully collected data for 202208
Successfully collected data for 202209
Successfully collected data for 202210
Successfully collected data for 202211
Successfully collected data for 202212
Successfully collected data for 202301
Successfully collected da

In [6]:
url = 'http://openapi.seoul.go.kr:8088/4a4e4a6b6672686b36374f65445347/json/CardSubwayTime/1/1000/202101/'

response = requests.get(url)
result = response.json()


In [12]:
im = result['CardSubwayTime']['row']
im = pd.DataFrame(im)
im['SBWY_ROUT_LN_NM'].unique()
stations = im['STTN'].unique()

locations = [
    "압구정", "신사", "논현", "학동", "청담", "봉은사", 
    "삼성중앙", "선정릉", "언주", "신논현", "역삼", "선릉", 
    "삼성(무역센터)", "학여울", "대청", "일원", "수서", "대치", "도곡", 
    "매봉", "양재(서초구청)", "대모산입구", "한티","압구정로데오",

    " 강남구청", "강남", "개포동", "구룡"
]

find_sts = []

for sts in stations:
    if sts in locations:
        find_sts.append(sts)
print(find_sts)
print(sorted(locations))

['삼성(무역센터)', '선릉', '역삼', '강남', '압구정', '신사', '양재(서초구청)', '매봉', '도곡', '대치', '학여울', '대청', '일원', '수서', '청담', '학동', '논현', '신논현', '언주', '선정릉', '삼성중앙', '봉은사', '한티', '구룡', '개포동', '대모산입구', '압구정로데오']
[' 강남구청', '강남', '개포동', '구룡', '논현', '대모산입구', '대청', '대치', '도곡', '매봉', '봉은사', '삼성(무역센터)', '삼성중앙', '선릉', '선정릉', '수서', '신논현', '신사', '압구정', '압구정로데오', '양재(서초구청)', '언주', '역삼', '일원', '청담', '학동', '학여울', '한티']


In [9]:
# find_sts에 포함된 역만 필터링
filtered_df = im[im['STTN'].isin(find_sts)]


# 결과 출력
print(filtered_df)
filtered_df.to_excel("강남지하철이용량.xlsx", index=True)


     USE_MM SBWY_ROUT_LN_NM      STTN  HR_4_GET_ON_NOPE  HR_4_GET_OFF_NOPE  \
28   202101             2호선  삼성(무역센터)             121.0                0.0   
29   202101             2호선        선릉              30.0                1.0   
30   202101             2호선        역삼              36.0                0.0   
31   202101             2호선        강남             110.0                3.0   
77   202101             3호선       압구정              51.0                5.0   
78   202101             3호선        신사              18.0                4.0   
83   202101             3호선  양재(서초구청)              42.0                0.0   
84   202101             3호선        매봉               7.0                0.0   
85   202101             3호선        도곡              23.0                6.0   
86   202101             3호선        대치               4.0                0.0   
87   202101             3호선       학여울               2.0                0.0   
88   202101             3호선        대청              58.0         

In [102]:
im.to_excel('강남지하철데이터.xlsx', index=True)

In [107]:
im['STTN'].duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
599    False
600     True
601    False
602    False
603    False
Name: STTN, Length: 604, dtype: bool

### 데이터 파일 훑어보기

In [14]:
ls

1강남지하철이용량_전체_2021-2024.xlsx
2024_강남_통행속도.xlsx
datatreat.ipynb
holidays_2021_2024_formatted.xlsx
병합된_서울시_교통량_2021.xlsx
병합된_서울시_교통량_2022.xlsx
병합된_서울시_교통량_2023.xlsx
병합된_서울시_교통량_2024.xlsx
행정동인구_강남구/


In [16]:
df = pd.read_excel("2024_강남_통행속도.xlsx")
df

TypeError: expected <class 'str'>

### 올릭픽대로 인근 역 이용량 데이터 추출

In [13]:
# 데이터 읽기
df = pd.read_excel("2강남지하철이용량_전체_2021-2024.xlsx")

df['STTN'].unique()

array(['삼성(무역센터)', '선릉', '역삼', '강남', '압구정', '신사', '양재(서초구청)', '매봉', '도곡',
       '대치', '학여울', '대청', '일원', '수서', '청담', '강남구청', '학동', '논현', '신논현',
       '언주', '선정릉', '삼성중앙', '봉은사', '한티', '구룡', '개포동', '대모산입구', '압구정로데오'],
      dtype=object)

In [16]:
def read_olympic(df):
    near_olym_stations = ['봉은사', '청담', '압구정', '압구정로데오', '신사']
    try:
        filtered_df = df[df['STTN'].isin(near_olym_stations)].copy()
        filtered_df.to_excel("올림픽대로_인근_역이용량[2021-2024].xlsx", index=True)
        print("올림픽대로 인근 역 엑셀 파일 저장 성공")
    except:
        print("데이터 저장 실패")
    
read_olympic(df)

올림픽대로 인근 역 엑셀 파일 저장 성공


In [ ]:
df = pd.read_excel("올림픽대로_인근_역이용량[2021-2024].xlsx")
df.info()
print(df['STTN'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 55 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0.1        240 non-null    int64 
 1   Unnamed: 0          240 non-null    int64 
 2   USE_MM              240 non-null    int64 
 3   SBWY_ROUT_LN_NM     240 non-null    object
 4   STTN                240 non-null    object
 5   HR_4_GET_ON_NOPE    240 non-null    int64 
 6   HR_4_GET_OFF_NOPE   240 non-null    int64 
 7   HR_5_GET_ON_NOPE    240 non-null    int64 
 8   HR_5_GET_OFF_NOPE   240 non-null    int64 
 9   HR_6_GET_ON_NOPE    240 non-null    int64 
 10  HR_6_GET_OFF_NOPE   240 non-null    int64 
 11  HR_7_GET_ON_NOPE    240 non-null    int64 
 12  HR_7_GET_OFF_NOPE   240 non-null    int64 
 13  HR_8_GET_ON_NOPE    240 non-null    int64 
 14  HR_8_GET_OFF_NOPE   240 non-null    int64 
 15  HR_9_GET_ON_NOPE    240 non-null    int64 
 16  HR_9_GET_OFF_NOPE   240 no

In [21]:
print(df.head())

   Unnamed: 0.1  Unnamed: 0  USE_MM SBWY_ROUT_LN_NM    STTN  HR_4_GET_ON_NOPE  \
0             4           4  202101             3호선     압구정                51   
1             5           5  202101             3호선      신사                18   
2            14          14  202101             7호선      청담                14   
3            22          22  202101        9호선2~3단계     봉은사                 0   
4            30          30  202101             분당선  압구정로데오                35   

   HR_4_GET_OFF_NOPE  HR_5_GET_ON_NOPE  HR_5_GET_OFF_NOPE  HR_6_GET_ON_NOPE  \
0                  5              2219               1821              6179   
1                  4              1850               1286              4389   
2                  0              1678               3281              5366   
3                  0              1003                974              3775   
4                  0              1050                695              2842   

   ...  HR_0_GET_ON_NOPE  HR_0_GET_OFF

In [7]:
speed22 = pd.read_excel("2022_강남_통행속도.xlsx")
speed22.head()

,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,20220101,토,강남대로,1220021100,우성아파트,강남역,상행,586,4,주간선도로,...,22.89,22.10,19.96,21.57,26.67,24.71,25.09,25.69,26.14,28.90
1,20220101,토,강남대로,1220016200,우성아파트,뱅뱅사거리,하행,414,4,주간선도로,...,30.25,26.91,28.42,32.13,31.95,33.38,32.12,31.55,22.26,21.55
2,20220101,토,개포로,1220020500,대청초교,대청역,상행,862,2,기타도로,...,23.14,25.01,23.62,22.86,23.79,25.81,24.97,26.80,26.15,24.87
3,20220101,토,개포로,1220018600,대청역,수서경찰서,상행,318,2,기타도로,...,30.71,26.13,27.78,27.90,27.75,27.79,26.54,26.24,29.38,27.38
4,20220101,토,개포로,1220017200,수서경찰서,대모산입구역,상행,333,2,기타도로,...,23.25,21.77,24.39,30.45,24.78,29.20,26.34,29.03,26.18,32.19
